In [2]:
# Scraping hashtag on Twitter

import json
import codecs
import csv
import tweepy
from tweepy import OAuthHandler

class UTF8Recoder:

    def __init__(self, f, encoding):
        self.reader = codecs.getreader(encoding)(f)

    def __iter__(self):
        return self

    def next(self):
        return self.reader.next().encode('utf-8')

class UnicodeReader:

    def __init__(self, f, dialect=csv.excel, encoding='utf-8', **kwds):
        f = UTF8Recoder(f, encoding)
        self.reader = csv.reader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return [unicode(s, 'utf-8') for s in row]

    def __iter__(self):
        return self

class UnicodeWriter:

    def __init__(self, f, dialect=csv.excel, encoding='utf-8', **kwds):
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode('utf-8') for s in row])
        data = self.queue.getvalue()
        data = data.decode('utf-8')
        data = self.encoder.encode(data)
        self.stream.write(data)
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# Carrega as credenciais da API do Twitter
# Nesse caso, as credenciais estão em um JSON salvo no mesmo diretório, que vai ser puxado pelo código
with open('credenciais_twitter.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['consumer_key']
    consumer_secret = info['consumer_secret']
    access_token = info['access_token']
    access_token_secret = info['access_token_secret']

# Autentica as chaves de acesso
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# max_tweets = 10 # Define número máximo de tweets que serão colocados

# Pergunta o termo de busca para o usuário
termo_de_busca = input("Qual o termo ou hashtag que você deseja buscar no twitter? ")
print()

# Escreve o CSV
with open('tweets_%s.csv' % termo_de_busca, 'w', encoding='utf-8') as f:
    wr = csv.writer(f, delimiter=';', lineterminator='\n')
    wr.writerow(["id", "datetime", "created_at", "tweet"])
    
    for tweet in tweepy.Cursor(api.search, q=termo_de_busca, tweet_mode='extended', lang = "pt", rpp = 100).items(): #items(max_tweets) para definir máximo de tweets
        output = [tweet.id, tweet.created_at, tweet.created_at.strftime("%d-%m-%Y %H:%M:%S"), tweet.full_text]
        wr.writerow(output)

print("Terminei!")

Qual o termo ou hashtag que você deseja buscar no twitter? #Oscars2019

Terminei!


In [5]:
# Scraping hashtag on Twitter

import json
import codecs
import csv
import tweepy
from tweepy import OAuthHandler

# Carrega as credenciais da API do Twitter
# Nesse caso, as credenciais estão em um JSON salvo no mesmo diretório, que vai ser puxado pelo código
with open('credenciais_twitter.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['consumer_key']
    consumer_secret = info['consumer_secret']
    access_token = info['access_token']
    access_token_secret = info['access_token_secret']

# Autentica as chaves de acesso
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Pergunta o termo de busca para o usuário
termo_de_busca = input("Qual o termo ou hashtag que você deseja buscar no twitter? ")
print()

all_tweets = []
new_tweets = tweepy.Cursor(api.search, q=termo_de_busca, tweet_mode="extended", lang="pt", rpp=100).items()
all_tweets.extend(new_tweets)

# Salva o id do tweet mais antigo menos 1
oldest = all_tweets[-1].id - 1

# Continua pegando os tweets até não ter mais
while len(new_tweets) > 0: # len só será 0 quando a request não retornar nada
    print(f"Pegando tweets antes de {oldest}")
    
    # Todas as requests seguintes usam o parâmetro max_id para evitar duplicatas
    new_tweets = tweepy.Cursor(api.search, q=termo_de_busca, tweet_mode="extended", lang="pt", rpp=100, max_id=oldest).items()
    
    # Salva os tweets mais recentes
    all_tweets.extend(new_tweets)
    
    # Atualiza o id do tweet mais antigo com id menos 1
    oldest = all_tweets[-1].id - 1

# transorma os tweets em uma lista que vai preencer o CSV
output = [[tweet.id, tweet.created_at, tweet.created_at.strftime("%d-%m-%Y %H:%M:%S"), tweet.full_text] for tweet in all_tweets]

# Escreve o CSV
with open('tweets_%s.csv' % termo_de_busca, 'w', encoding='utf-8') as f:
    wr = csv.writer(f, delimiter=';', lineterminator='\n')
    wr.writerow(["id", "datetime", "created_at", "tweet"])
    wr.writerows(output)

print("Terminei!")
# print(tweet.created_at, tweet.text)

Qual o termo ou hashtag que você deseja buscar no twitter? #Oscars2019



TypeError: object of type 'ItemIterator' has no len()